In [1]:
import os
import tensorflow as tf
tf.enable_eager_execution()
import pandas as pd
import math
import numpy as np
import itertools
import collections

In [2]:
from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

In [3]:
# get all segment file names and save in Filenames
def scan(folder):        
    x=[]
    for path,dirs,files in os.walk(folder):
        for f in files:
            if f.startswith('segment'):
                x.append(os.path.join(path,f))
    return(x)

folder = './waymo-od/data'

Filenames=scan(folder)

In [4]:
def getID(f):         #get the ID of segment from file name
    f = f.replace('_with_camera_labels.tfrecord', '')
    f = f.split('segment-',1)[1]
    return f

In [201]:
### Extract segment files in Frames, each file consists of multiple frames

Frames = {}

for file in Filenames[0:3]:
    SegID = getID(file)
    Segment = []
    dataset = tf.data.TFRecordDataset(file, compression_type='')
    for data in dataset:
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        Segment.append(frame)
    Frames[SegID] = Segment

## Each frame includes:
#### 1. ***context***: similar for all the frames in one segment
#### 2. ***timestamp_micros*** : should be devided to 1000000
#### 3. ***pose***
#### 4. ***laser_labels*** : includes
#### 5. ***camera_labels***

In [202]:
def CountLLabels(frame):    #number of laser labels in a frame
    return len(frame.laser_labels)

def converttime(t):
    t = t/1000000
    t = datetime.fromtimestamp(t)
    return t

In [203]:
def Ped2df(SegID, tstamp, pedLoc, 
           pedMeta,VehPos,tday,w):
    
    length = len(pedLoc)
    t, px, py, pz, pw, pl, pht, phd, psx, psy, paccx, paccy, vpose = (
        [] for i in range(13)
    )
    
    for i in range(length):
        t.append(tstamp[i])
        px.append(pedLoc[i].center_x)
        py.append(pedLoc[i].center_y)
        pz.append(pedLoc[i].center_z)
        pw.append(pedLoc[i].width)       
        pl.append(pedLoc[i].length)
        pht.append(pedLoc[i].height)
        phd.append(pedLoc[i].heading)
        psx.append(pedMeta[i].speed_x)
        psy.append(pedMeta[i].speed_y)
        paccx.append(pedMeta[i].accel_x)
        paccy.append(pedMeta[i].accel_y)
        vpose.append(VehPos[i].transform)
        
    data = np.array([t, px, py, pz, pw, pl, pht, phd,
                     psx, psy, paccx, paccy, vpose]).T.tolist()
    columnsnames= ['Timestamp', 'x', 'y', 
              'z', 'Width', 'Length',
              'Height','Heading', 'Speed x',
              'Speed y', 'accel x', 'accel y', 'vpose']
    pedDF = pd.DataFrame(data, columns = columnsnames)
    
    transform = pedDF['vpose'].apply(pd.Series)
    transform = transform.rename(columns= lambda x: 'AVtransform_' +str(x))
    pedDF = pedDF.drop(columns= 'vpose')
    pedDF = pd.concat([pedDF[:],transform[:]],axis=1)
    
    pedDF['Weather'] = w
    pedDF['Time of Day'] = tday
    pedDF['Segment ID'] = SegID
    return pedDF
 

In [204]:
def PedExtract(SegID):
    pedLoc = collections.defaultdict(list)                             #pedestrians label box
    pedMeta = collections.defaultdict(list)                            #pedestrians label metadata
    VehPos =  collections.defaultdict(list)                            #ego veh pose
    tstamp = collections.defaultdict(list) 
    tday = Frames[SegID][0].context.stats.time_of_day
    w = Frames[SegID][0].context.stats.weather
    
    for j in range(len(Frames[SegID])):
        t = Frames[SegID][j].timestamp_micros                      #timestamp
        vpos = Frames[SegID][j].pose                                #Vehicle position
        Nlabels = CountLLabels(Frames[SegID][j])                   # number of laser labels
       
        for k in range(Nlabels):
            LidarLab = Frames[SegID][j].laser_labels[k]
            if  LidarLab.type == 2:                      # type 2: pedestrian
                
                PedID = LidarLab.id

                tstamp[PedID].append(t)
                pedLoc[PedID].append(LidarLab.box)

                
                pedMeta[PedID].append(LidarLab.metadata)
                
                
                VehPos[PedID].append(Frames[SegID][j].pose)
    
    Ped = {}
    for p in list(pedLoc.keys()):
        Ped[p] = Ped2df(SegID, tstamp[p], pedLoc[p],
                        pedMeta[p],VehPos[p],tday, w)
        
    
    return Ped      

In [205]:
def MultiFrame(Frames):
    PedTraj = {}
    for i in range(len(Frames)):
        SegID = list(Frames.keys())[i]
        PedTraj.update(PedExtract(SegID))
    return PedTraj
    

In [206]:
x= MultiFrame(Frames)

In [217]:
(Frames).keys()

dict_keys(['10153695247769592104_787_000_807_000', '10231929575853664160_1160_000_1180_000', '10075870402459732738_1060_000_1080_000'])

In [221]:
x = PedExtract('10075870402459732738_1060_000_1080_000')

In [223]:
x['4kOQSGT-rxdpEq7DRZV8_w']

,Timestamp,x,y,z,Width,Length,Height,Heading,Speed x,Speed y,...,AVtransform_9,AVtransform_10,AVtransform_11,AVtransform_12,AVtransform_13,AVtransform_14,AVtransform_15,Weather,Time of Day,Segment ID
0,1550088692199980,4.764756,27.298530,-0.000533,1.170691,1.076253,1.75,1.463004,-0.204482,-1.281589,...,0.013627,0.993908,19.066,0.0,0.0,0.0,1.0,sunny,Day,10075870402459732738_1060_000_1080_000
1,1550088692299728,4.929060,27.348042,-0.022655,1.170691,1.076253,1.75,1.446542,-0.193940,-1.289159,...,0.012676,0.994046,19.101,0.0,0.0,0.0,1.0,sunny,Day,10075870402459732738_1060_000_1080_000
2,1550088692399487,5.132779,27.386777,-0.056671,1.170691,1.076253,1.75,1.428509,-0.183398,-1.296730,...,0.012202,0.994298,19.137,0.0,0.0,0.0,1.0,sunny,Day,10075870402459732738_1060_000_1080_000
3,1550088692499172,5.367659,27.416508,-0.054402,1.170691,1.076253,1.75,1.409378,-0.172856,-1.304301,...,0.010298,0.994561,19.174,0.0,0.0,0.0,1.0,sunny,Day,10075870402459732738_1060_000_1080_000
4,1550088692598889,5.638499,27.430831,-0.093966,1.170691,1.076253,1.75,1.388654,-0.162314,-1.311871,...,0.009824,0.994782,19.210,0.0,0.0,0.0,1.0,sunny,Day,10075870402459732738_1060_000_1080_000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,1550088711575662,-60.035655,13.156489,-0.161381,1.170691,1.076253,1.75,1.313509,0.668872,-0.297106,...,0.010822,0.998869,16.906,0.0,0.0,0.0,1.0,sunny,Day,10075870402459732738_1060_000_1080_000
195,1550088711675573,-60.289802,13.649958,-0.318741,1.170691,1.076253,1.75,1.330994,0.701826,-0.358665,...,0.009767,0.998748,16.893,0.0,0.0,0.0,1.0,sunny,Day,10075870402459732738_1060_000_1080_000
196,1550088711775479,-60.542833,14.135143,-0.395961,1.170691,1.076253,1.75,1.348578,0.734779,-0.420225,...,0.010628,0.998687,16.877,0.0,0.0,0.0,1.0,sunny,Day,10075870402459732738_1060_000_1080_000
197,1550088711875379,-60.682243,14.625051,-0.379912,1.170691,1.076253,1.75,1.365778,0.767733,-0.481784,...,0.010724,0.998695,16.859,0.0,0.0,0.0,1.0,sunny,Day,10075870402459732738_1060_000_1080_000
